In [1]:
import numpy as np

from pprint import pprint
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from scipy.spatial.distance import cosine

import tensorflow as tf

In [2]:
N = 2000
dim = 5
theta = np.pi/8
np.random.seed(0)

vecs = np.random.rand(N, dim)
dists = cosine_similarity(vecs, vecs)
cond = dists > np.cos(theta)
ixs = np.triu_indices(N, k=1)

#################################################################

data = np.hstack((vecs[ixs[0]], vecs[ixs[1]]))
#data = np.subtract(vecs[ixs[0]],  vecs[ixs[1]])
#data = np.reshape(1 - dists[ixs], (len(ixs[0]), -1))

#################################################################

data_y = cond[ixs].astype(int)
un = np.unique(data_y, return_counts=True)
print(un)
counts = un[1]
print('baseline: %s\n' % (counts[0]/sum(counts)))

X_train, X_test, y_train, y_test = train_test_split(
    data, data_y, test_size=0.33, random_state=42, stratify=data_y)

if data.shape[1] > 1:
    norm = Normalizer()
    X_train = norm.fit_transform(X_train)
    X_test = norm.transform(X_test)

def gen_batches(X, y, batch_size=100):
    for i in range(0, X.shape[0], batch_size):
        yield X[i:i+batch_size], y[i:i+batch_size]


(array([0, 1]), array([1765742,  233258]))
baseline: 0.8833126563281641



In [3]:
learning_rate = 0.001
batch_size = 128
num_steps = int(X_train.shape[0]/batch_size) + 1

In [4]:
# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=data.shape[1])]

# Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[128, 128],
    optimizer=tf.train.AdamOptimizer(learning_rate),
    n_classes=2,
    loss_reduction=tf.losses.Reduction.MEAN,
    dropout=0.1,
    #model_dir="./tmp/dnn_model"
)

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    batch_size=batch_size,
    shuffle=True 
)

classifier.train(input_fn=train_input_fn, steps=num_steps)

# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
test_eval = classifier.evaluate(input_fn=test_input_fn)
print("\ntest metrics:\n")
pprint(test_eval)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j_/x1m_zvbn7dn_d4dkhl87p2mc0000gn/T/tmpkg9ekoy3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12ec4f7b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for 

INFO:tensorflow:global_step/sec: 677.848
INFO:tensorflow:loss = 0.083278716, step = 5901 (0.148 sec)
INFO:tensorflow:global_step/sec: 706.519
INFO:tensorflow:loss = 0.05832093, step = 6001 (0.142 sec)
INFO:tensorflow:global_step/sec: 566.136
INFO:tensorflow:loss = 0.07730112, step = 6101 (0.177 sec)
INFO:tensorflow:global_step/sec: 646.371
INFO:tensorflow:loss = 0.05348447, step = 6201 (0.155 sec)
INFO:tensorflow:global_step/sec: 695.874
INFO:tensorflow:loss = 0.05662231, step = 6301 (0.144 sec)
INFO:tensorflow:global_step/sec: 678.127
INFO:tensorflow:loss = 0.06632211, step = 6401 (0.147 sec)
INFO:tensorflow:global_step/sec: 683.607
INFO:tensorflow:loss = 0.0874871, step = 6501 (0.146 sec)
INFO:tensorflow:global_step/sec: 672.123
INFO:tensorflow:loss = 0.12294154, step = 6601 (0.149 sec)
INFO:tensorflow:global_step/sec: 662.357
INFO:tensorflow:loss = 0.04258571, step = 6701 (0.151 sec)
INFO:tensorflow:global_step/sec: 676.541
INFO:tensorflow:loss = 0.0432211, step = 6801 (0.148 sec)
I